# Background replacement using a mask image

One way to replace the background of an image is to provide a user-created mask image defining what should be considered background. This notebook demonstrates that approach.

The script below will replace everything except the center flower pot in the image on the left by using the mask image in the center. The output will look something like the image on the right.

<div style="display: flex; justify-content: space-between;">
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/three_pots.jpg" width="100%" style="padding: 4px">
            <br>
            <em>Input image</em>
        </p>
    </div>
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/three_pots-center_pot_mask.png" width="100%" style="padding: 4px">
            <br>
            <em>Mask</em>
        </p>
    </div>
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/doc-images/background-with-mask-image-example-1.png" width="100%" style="padding: 4px">
            <br>
            <em>Output image</em>
        </p>
    </div>
</div>

In [ ]:
from random import randint
from amazon_image_gen import BedrockImageGenerator
import file_utils
import logging
import base64
from datetime import datetime

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')

# Set the image to be edited.
source_image = "../images/three_pots.jpg"

# Set the image to be used as a mask. IMPORTANT: The mask must be in PNG format and must contain only pure
# black and pure white pixels.
mask_image = "../images/three_pots-center_pot_mask.png"

# Load the source image from disk.
with open(source_image, "rb") as image_file:
    source_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

with open(mask_image, "rb") as mask_file:
    mask_image_base64 = base64.b64encode(mask_file.read()).decode("utf-8")

# Configure the inference parameters.
inference_params = {
    "taskType": "OUTPAINTING",
    "outPaintingParams": {
        "image": source_image_base64,
        "text": "potted flower sitting on a picnic table, backyard wedding decorations, beautiful, highest quality",  # Description of the background to generate
        "maskImage": mask_image_base64,
        "outPaintingMode": "PRECISE",  # "DEFAULT" softens the mask. "PRECISE" keeps it sharp.
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,  # Number of variations to generate. 1 to 5.
        "quality": "standard",  # Allowed values are "standard" and "premium"
        "cfgScale": 7.0,  # How closely the prompt will be followed
        "seed": randint(0, 858993459),  # Use a random seed
    },
}

# Define an output directory with a unique name.
generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_directory = f"output/{generation_id}"

# Create the generator.
generator = BedrockImageGenerator(
    output_directory=output_directory
)

# Generate the image(s).
response = generator.generate_images(inference_params)

if "images" in response:
    # Save and display each image
    images = file_utils.save_base64_images(response["images"], output_directory, "image")
    for image in images:
        display(image)